**Metadata RAG Workflow**

By working through this Jupyter Notebook together, we'll gain hands-on experience in using LangChain to generate column descriptions for rows with missing values in a dataset. We'll learn how to leverage the LlamaCpp language model and few-shot learning techniques to generate descriptions based on semantically similar examples.

Let's dive in and start generating those column descriptions!


Add the following packages

In [1]:
# !pip install langchain-community sentence-transformers pandas faiss-cpu openpyxl transformers datasets peft
# !pip install accelerate bitsandbytes trl safetensors lm-eval gradio flask pgvector sentence-transformers langchain psycopg2-binary tiktoken openai pypdf



We'll start by importing the necessary libraries from LangChain, LangChain Community, and pandas.

In [2]:
# Import necessary libraries
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
#from langchain_community.llms import CTransformers, LlamaCpp <-- For open source runnning on CPU LlamaCppp = GGUF models, CTransformers .bin
from sentence_transformers import SentenceTransformer, util
import pandas as pd

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

Adding a cosine similarity function to compare the similarity between two strings

In [3]:
def similarity(reference: str, prediction: str) -> str:
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    embedding_1 = model.encode(reference, convert_to_tensor=True)
    embedding_2 = model.encode(prediction, convert_to_tensor=True)
    similarity_score = util.pytorch_cos_sim(embedding_1, embedding_2).item()
    return f"{similarity_score:.2f}"

We'll initialize the LlamaCpp language model with the specified parameters, such as the model path, temperature, maximum tokens, and batch size.

In [4]:
# THE FIRST TIME YOU RUN THIS, IT MIGHT TAKE A WHILE

model_path_or_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_path_or_id)
model = AutoModelForCausalLM.from_pretrained(
    model_path_or_id,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    bnb_4bit_compute_dtype=torch.float16,
    #use_flash_attention_2=True,
    attn_implementation="flash_attention_2",
    load_in_4bit=True
)

def llm(prompt):
    """Convenience function for generating model output"""
    # Tokenize the input
    input_ids = tokenizer(
        prompt, 
        return_tensors="pt", 
        truncation=True).input_ids.cuda()
    
    # Generate new tokens based on the prompt, up to max_new_tokens
    # Sample aacording to the parameter
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=input_ids, 
            max_new_tokens=20, 
            do_sample=True, 
            top_p=0.9,
            temperature=0.1,
            use_cache=True
        )
    return tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

# Initialize the LlamaCpp language model
#llm = LlamaCpp(
#    model_path="models/phi-2.Q5_K_M.gguf",
#    temperature=0.01,
#    max_tokens=30,
#    n_gpu_layers=-1,
#    n_batch=512,
#    f16_kv=True,
#    verbose=False,
#)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 28.00 MiB. GPU 

We'll define an example prompt template using the PromptTemplate class, which specifies the input variables and the template string.

In [ ]:

# Define the example prompt template
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)


We'll load the metadata from an Excel file using pandas and display the first 5 rows of the DataFrame to get a glimpse of the data.

In [ ]:

# Load the metadata from an Excel file
metadata = "data/demo_excel.xlsx"
df = pd.read_excel(metadata)
df.head(5)


We'll drop rows with missing values from the DataFrame to create a complete dataset that we can use as examples.

In [ ]:

# Drop rows with missing values
df_complete = df.dropna()

We'll create a list of examples from the complete rows by applying a lambda function to each row. The examples will consist of the column name, full name, table name, and data type as input, and the column description as output

In [5]:
# Create a list of examples from the complete rows
examples = df_complete.apply(
    lambda row: {
        "input": f"{row['TABLE_NAME']},{row['COLUMN_NAME']},{row['COLUMN_FULL_NAME']},{row['DATA_TYPE']}",
        "output": row["COLUMN_DESCRIPTION"],
    },
    axis=1,
).tolist()


NameError: name 'df_complete' is not defined

We'll initialize the SemanticSimilarityExampleSelector using the examples, embeddings, and vectorstore class. This selector will help us find the most similar examples based on semantic similarity.

In [ ]:

# Initialize the SemanticSimilarityExampleSelector
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=HuggingFaceEmbeddings(
        model_name="all-MiniLM-L6-v2",
        model_kwargs={"device": "cpu"},
    ),
    vectorstore_cls=FAISS,
    k=5,
)


We'll create a few-shot prompt template using the FewShotPromptTemplate class. This template will include the example selector, example prompt, prefix, suffix, and input variables.

In [ ]:

# Create the few-shot prompt template
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="You are a column description generator. Given the following Examples below ",
    suffix="predict the Output for the given input \nInput: {metadata}\nOutput:",
    input_variables=["metadata"],
)


We'll select the rows with missing values from the original DataFrame to focus on the rows that need column descriptions.

In [ ]:

# Select rows with missing values
df_empty = df[df.isnull().any(axis=1)]


We'll iterate over the first 5 rows with missing values to generate column descriptions for them. For each row, we'll create a pre-prompt string using the column name, full name, and data type. We'll format the few-shot prompt template with the pre-prompt string to create a complete prompt. We'll print the generated prompt to see how it looks.

In [ ]:

for i, row in df_empty.head(1).iterrows(): #We iterate over .head(x) rows

    #Here we collect the examples we want to parse for the output on top of the k-examples
    pre_prompt = f"{row['TABLE_NAME']},{row['COLUMN_NAME']},{row['COLUMN_FULL_NAME']},{row['DATA_TYPE']}"

    #Now we load the target metadata into similar_prompt
    prompt = similar_prompt.format(metadata=pre_prompt)
    
    #Here what the final prompt looks like 
    print(f"\n\nPROMPT:\n{prompt} \n")

    #Here we parse the prompt to the LLM and receive the models repsonse
    response = llm.invoke(prompt)
    print("Model Response: " , response)

    #Here we compare the response we want, with the model response using cosine similarity
    sim = similarity("This column contains the name of the Lakeside", response)
    print(f"Similarity Score (0 - Bad, 1 - Perfect Match): {sim} \n")


We'll invoke the LlamaCpp language model with the generated prompt to generate the column description.

Finally, we'll print the generated column description to see the result.

In [ ]:
'''similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="You are a column description generator. Given the following examples of Table Name, Column Name, Column Full Name and Data Type to Column Description below",
    suffix="Predict the Column Description for the given Table Name, Column Name, Column Full Name and Data Type \nInput: {metadata}\nOutput:",
    input_variables=["metadata"],
)'''

**Noticed its going abit wrong?** 
Play around with the prompt template (example that worked for me above), model paramaters and K-samples 